Add intro here

### 1 Text to Speech

In [1]:
!pip install transformers
!pip install gradio
!pip install timm
!pip install timm
!pip install inflect
!pip install phonemizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:0031m109.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.0 MB/s eta 0:00:00
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
INFO: pip is looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 19.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.1 MB/s eta 0:00:00
Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
from transformers.utils import logging

logging.set_verbosity_error()

/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import pipeline

narrator = pipeline("text-to-speech",
                    model="kakao-enterprise/vits-ljs")

/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
text = """
Researchers at the Allen Institute for AI, \
HuggingFace, Microsoft, the University of Washington, \
Carnegie Mellon University, and the Hebrew University of \
Jerusalem developed a tool that measures atmospheric \
carbon emitted by cloud servers while training machine \
learning models. After a model’s size, the biggest variables \
were the server’s location and time of day it was active.
"""

In [6]:
narrated_text = narrator(text)

In [7]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(narrated_text["audio"][0],
             rate=narrated_text["sampling_rate"])

## 2 Story Generation

In [10]:
#!pip install -U langchain-ollama

In [11]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1")

In [12]:
response = llm.invoke("Can you write a very short story of 7 sentences for 3 year old kid?")
print(response)

Here is a short story for a 3-year-old:

Benny the bunny loved to play outside. He had a big garden with lots of flowers and a little pond with fish. One sunny day, Benny decided to go on an adventure. He picked some juicy carrots from his garden and put them in a basket. Then he went to visit his friend Emma, who was playing by the pond. They shared their snacks and watched the fish swim together. Benny said "I'm so happy!" and gave Emma a big hug!


## 3 Text to Video

In [1]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To avoid PEFT backend required error:

(As of 1 April 2024) Ensure your library has the following versions:

    peft >= 0.6.0
    transformers >= 4.34.0


In [2]:
import transformers
transformers.__version__

'4.38.2'

In [3]:
#!pip install peft
import peft
peft.__version__

'0.12.0'

In [4]:
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  6.63it/s]


In [5]:
output = pipe(
    prompt="A space rocket with trails of smoke behind it launching into space from the desert, 4k, high resolution",
    negative_prompt="bad quality, worse quality, low resolution",
    num_frames=16,
    guidance_scale=2.0,
    num_inference_steps=6,
    generator=torch.Generator("cpu").manual_seed(0),
)
frames = output.frames[0]
export_to_gif(frames, "animatelcm.gif")

100%|██████████| 6/6 [00:13<00:00,  2.22s/it]


'animatelcm.gif'

In [8]:
#!pip install ipywidgets

In [9]:
import ipywidgets as widgets
display(widgets.HTML(f'<img src="{animatelcm.gif}" width="750" align="center">'))

NameError: name 'animatelcm' is not defined

In [10]:
!ls

animatelcm.gif		    fastai     LlaVa
astronaut_rides_horse2.png  fastai2    my_github
astronaut_rides_horse.png   fasttext   output_file
clinical_bert		    llama.cpp  output_file_cart_generations


In [11]:
!pwd

/home/ubuntu/codebase
